# Dimensional Modeling

Dimensional modeling is a data warehouse design technique that organizes data into **facts** and **dimensions** to optimize query performance and business understanding. It was pioneered by **Ralph Kimball** and remains the foundation of modern analytical data warehousing.

---

## 📚 Table of Contents

1. [Kimball Methodology](#kimball-methodology)
2. [Fact Tables](#fact-tables)
3. [Dimension Tables & SCD Types](#dimension-tables--scd-types)
4. [Conformed & Junk Dimensions](#conformed--junk-dimensions)
5. [Building a Star Schema with Python](#building-a-star-schema-with-python)
6. [Key Takeaways](#key-takeaways)

---

## Kimball Methodology

The **Kimball methodology** (also called the **Bottom-Up approach**) is a data warehouse design approach that focuses on delivering business value quickly through iterative development of dimensional data marts.

### Core Principles

| Principle | Description |
|-----------|-------------|
| **Focus on Business Process** | Design data marts around specific business processes (sales, inventory, etc.) |
| **Dimensional Modeling** | Use star/snowflake schemas with facts and dimensions |
| **Conformed Dimensions** | Share dimension tables across data marts for consistency |
| **Iterative Delivery** | Build incrementally, delivering value with each data mart |
| **Bus Architecture** | Define enterprise-wide dimensions and facts for integration |

### Four-Step Dimensional Design Process

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     Kimball's Design Process                            │
├─────────────────────────────────────────────────────────────────────────┤
│  Step 1: Select the Business Process                                    │
│          └── What process are we modeling? (e.g., retail sales)         │
│                                                                         │
│  Step 2: Declare the Grain                                              │
│          └── What does one row in the fact table represent?             │
│              (e.g., one line item in a sales transaction)               │
│                                                                         │
│  Step 3: Identify the Dimensions                                        │
│          └── Who, What, When, Where, Why, How?                          │
│              (e.g., Date, Product, Store, Customer)                     │
│                                                                         │
│  Step 4: Identify the Facts                                             │
│          └── What are we measuring?                                     │
│              (e.g., quantity_sold, unit_price, discount_amount)         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Star Schema vs Snowflake Schema

| Aspect | Star Schema | Snowflake Schema |
|--------|-------------|------------------|
| **Structure** | Denormalized dimensions | Normalized dimensions |
| **Query Performance** | Faster (fewer joins) | Slower (more joins) |
| **Storage** | More redundancy | Less redundancy |
| **Complexity** | Simpler | More complex |
| **Maintenance** | Easier | Harder |

---

## Fact Tables

A **fact table** stores quantitative data (measurements/metrics) about a business process. It contains:
- **Foreign keys** to dimension tables
- **Measures/Facts** (numeric values that can be aggregated)

### Types of Fact Tables

#### 1. Transactional Fact Table

Records **individual events** at the most granular level. Each row represents one transaction.

```
┌─────────────────────────────────────────────────────────────────┐
│              FACT_SALES (Transactional)                         │
├─────────────────────────────────────────────────────────────────┤
│  sale_key (PK)                                                  │
│  date_key (FK) ─────────────────► DIM_DATE                      │
│  product_key (FK) ──────────────► DIM_PRODUCT                   │
│  customer_key (FK) ─────────────► DIM_CUSTOMER                  │
│  store_key (FK) ────────────────► DIM_STORE                     │
│  ───────────────────────────────────────────────────────────    │
│  quantity_sold          (Additive)                              │
│  unit_price             (Non-Additive)                          │
│  discount_amount        (Additive)                              │
│  sales_amount           (Additive)                              │
└─────────────────────────────────────────────────────────────────┘
```

**Characteristics:**
- One row per transaction/event
- Highly granular
- Can grow very large
- Sparse (many dimension combinations may never occur)

---

#### 2. Periodic Snapshot Fact Table

Captures the **state of measures at regular intervals** (daily, weekly, monthly).

```
┌─────────────────────────────────────────────────────────────────┐
│           FACT_INVENTORY_SNAPSHOT (Periodic)                    │
├─────────────────────────────────────────────────────────────────┤
│  snapshot_key (PK)                                              │
│  date_key (FK) ─────────────────► DIM_DATE (end of period)      │
│  product_key (FK) ──────────────► DIM_PRODUCT                   │
│  warehouse_key (FK) ────────────► DIM_WAREHOUSE                 │
│  ───────────────────────────────────────────────────────────    │
│  quantity_on_hand       (Semi-Additive)                         │
│  quantity_on_order      (Semi-Additive)                         │
│  inventory_value        (Semi-Additive)                         │
│  days_of_supply         (Non-Additive)                          │
└─────────────────────────────────────────────────────────────────┘
```

**Characteristics:**
- One row per entity per time period
- Dense (every combination has a row for each period)
- Contains **semi-additive** facts (can sum across some dimensions, but not time)
- Good for trending analysis

---

#### 3. Accumulating Snapshot Fact Table

Tracks the **lifecycle of a process** with multiple date milestones. Rows are **updated** as the process progresses.

```
┌─────────────────────────────────────────────────────────────────┐
│           FACT_ORDER_FULFILLMENT (Accumulating)                 │
├─────────────────────────────────────────────────────────────────┤
│  order_key (PK)                                                 │
│  order_date_key (FK) ───────────► DIM_DATE                      │
│  ship_date_key (FK) ────────────► DIM_DATE                      │
│  delivery_date_key (FK) ────────► DIM_DATE                      │
│  payment_date_key (FK) ─────────► DIM_DATE                      │
│  product_key (FK) ──────────────► DIM_PRODUCT                   │
│  customer_key (FK) ─────────────► DIM_CUSTOMER                  │
│  ───────────────────────────────────────────────────────────    │
│  order_amount                                                   │
│  days_to_ship           (calculated lag)                        │
│  days_to_deliver        (calculated lag)                        │
│  days_to_payment        (calculated lag)                        │
└─────────────────────────────────────────────────────────────────┘
```

**Characteristics:**
- One row per process instance (e.g., one order)
- Multiple date foreign keys representing milestones
- Rows are **updated** (not append-only)
- Includes lag metrics between milestones

---

### Fact Table Comparison

| Aspect | Transactional | Periodic Snapshot | Accumulating Snapshot |
|--------|---------------|-------------------|----------------------|
| **Grain** | One event | One entity per period | One process lifecycle |
| **Updates** | Insert only | Insert only | Updates allowed |
| **Date FKs** | One | One (period end) | Multiple (milestones) |
| **Density** | Sparse | Dense | Sparse |
| **Use Case** | Event analysis | State trending | Process tracking |

---

## Dimension Tables & SCD Types

**Dimension tables** provide the descriptive context (the "who, what, where, when, why, how") for fact table measurements.

### Dimension Table Characteristics

- Contains a **surrogate key** (synthetic primary key)
- Contains a **natural key** (business identifier)
- Wide tables with many descriptive attributes
- Relatively small compared to fact tables
- Often denormalized for query performance

```
┌─────────────────────────────────────────────────────────────────┐
│                      DIM_PRODUCT                                │
├─────────────────────────────────────────────────────────────────┤
│  product_key (PK)          ◄── Surrogate Key                    │
│  product_id                ◄── Natural Key                      │
│  product_name                                                   │
│  product_description                                            │
│  brand_name                                                     │
│  category_name                                                  │
│  subcategory_name                                               │
│  department_name                                                │
│  unit_cost                                                      │
│  unit_price                                                     │
│  is_active                                                      │
│  effective_date                                                 │
│  expiration_date                                                │
└─────────────────────────────────────────────────────────────────┘
```

---

### Slowly Changing Dimensions (SCD)

Dimension attributes can change over time. **SCD types** define how to handle these changes.

#### SCD Type 0 - Retain Original

**Never update** the dimension attribute. Original value is preserved forever.

```
Use Case: Original customer signup date, SSN, birth date
```

---

#### SCD Type 1 - Overwrite

**Overwrite** the old value with the new value. No history preserved.

```
Before:  customer_key=1, name="John Smith", city="Boston"
After:   customer_key=1, name="John Smith", city="New York"  ← overwritten
```

**Pros:** Simple, no storage overhead  
**Cons:** Loses history

---

#### SCD Type 2 - Add New Row (Full History)

**Create a new row** with a new surrogate key. Track validity with date columns.

```
┌──────────────┬─────────────┬───────────┬────────────────┬────────────────┬───────────┐
│ customer_key │ customer_id │ city      │ effective_date │ expiration_date│ is_current│
├──────────────┼─────────────┼───────────┼────────────────┼────────────────┼───────────┤
│ 1            │ C001        │ Boston    │ 2020-01-01     │ 2023-06-30     │ N         │
│ 2            │ C001        │ New York  │ 2023-07-01     │ 9999-12-31     │ Y         │
└──────────────┴─────────────┴───────────┴────────────────┴────────────────┴───────────┘
```

**Pros:** Full history preserved, accurate historical reporting  
**Cons:** More complex queries, storage growth

---

#### SCD Type 3 - Add New Column

**Add a new column** to track the previous value. Limited history (usually just one prior value).

```
┌──────────────┬───────────────┬──────────────────┐
│ customer_key │ current_city  │ previous_city    │
├──────────────┼───────────────┼──────────────────┤
│ 1            │ New York      │ Boston           │
└──────────────┴───────────────┴──────────────────┘
```

**Pros:** Simple queries, easy comparison  
**Cons:** Limited history (only one previous value)

---

#### SCD Type 4 - Mini-Dimension

Split **rapidly changing attributes** into a separate mini-dimension.

```
                                  ┌──────────────────────┐
                                  │  DIM_CUSTOMER_DEMO   │
                                  │  (Mini-Dimension)    │
┌────────────────────┐            ├──────────────────────┤
│   DIM_CUSTOMER     │            │  demo_key (PK)       │
├────────────────────┤            │  age_band            │
│  customer_key (PK) │            │  income_band         │
│  customer_id       │            │  loyalty_tier        │
│  customer_name     │            └──────────────────────┘
│  email             │                      ▲
└────────────────────┘                      │
            ▲                               │
            │         ┌─────────────────────┴───────────────┐
            └─────────┤         FACT_SALES                  │
                      │  customer_key (FK)                  │
                      │  demo_key (FK)                      │
                      └─────────────────────────────────────┘
```

---

#### SCD Type 6 - Hybrid (1 + 2 + 3)

Combines Type 1, 2, and 3: new rows for history, plus a column for the current value.

```
┌──────────────┬─────────────┬───────────┬────────────────┬────────────────┬──────────────┐
│ customer_key │ customer_id │ city      │ current_city   │ effective_date │ is_current   │
├──────────────┼─────────────┼───────────┼────────────────┼────────────────┼──────────────┤
│ 1            │ C001        │ Boston    │ New York       │ 2020-01-01     │ N            │
│ 2            │ C001        │ New York  │ New York       │ 2023-07-01     │ Y            │
└──────────────┴─────────────┴───────────┴────────────────┴────────────────┴──────────────┘
```

---

### SCD Types Summary

| Type | Strategy | History | Complexity | Use Case |
|------|----------|---------|------------|----------|
| **0** | Retain original | None | Low | Fixed attributes (SSN, DOB) |
| **1** | Overwrite | None | Low | Corrections, non-critical changes |
| **2** | New row | Full | High | Important business attributes |
| **3** | New column | Limited | Medium | "Before/After" analysis |
| **4** | Mini-dimension | Separate | Medium | Rapidly changing demographics |
| **6** | Hybrid | Full + Current | High | Best of all worlds |

---

## Conformed & Junk Dimensions

### Conformed Dimensions

A **conformed dimension** is shared across multiple fact tables/data marts with **identical structure and content**. This enables consistent analysis across business processes.

```
┌─────────────────────────┐
│      DIM_DATE           │
│   (Conformed)           │
├─────────────────────────┤
│  date_key (PK)          │
│  full_date              │
│  day_of_week            │
│  month_name             │
│  quarter                │
│  fiscal_year            │
└────────────┬────────────┘
             │
     ┌───────┴───────┬─────────────────┐
     ▼               ▼                 ▼
┌──────────┐   ┌──────────┐      ┌──────────┐
│FACT_SALES│   │FACT_INV  │      │FACT_SHIP │
│          │   │_SNAPSHOT │      │MENTS     │
└──────────┘   └──────────┘      └──────────┘
```

#### Benefits of Conformed Dimensions

- **Consistent reporting** across the enterprise
- **Drill-across queries** between fact tables
- **Reduced ETL complexity** (build once, use everywhere)
- **Single source of truth** for dimension data

#### Common Conformed Dimensions

| Dimension | Description |
|-----------|-------------|
| **Date/Time** | Calendar attributes, fiscal periods, holidays |
| **Customer** | Customer demographics across all touchpoints |
| **Product** | Product hierarchy used in sales, inventory, returns |
| **Geography** | Location hierarchy for all regional analysis |
| **Employee** | Staff information for HR, sales, operations |

---

### Junk Dimensions

A **junk dimension** combines multiple **low-cardinality flags and indicators** into a single dimension table, reducing fact table width and foreign key count.

#### Before Junk Dimension

```
┌─────────────────────────────────────────────────────────────────┐
│                    FACT_ORDERS (Before)                         │
├─────────────────────────────────────────────────────────────────┤
│  order_key                                                      │
│  date_key (FK)                                                  │
│  customer_key (FK)                                              │
│  is_rush_order         (Y/N)  ◄── Flag                          │
│  is_gift_wrapped       (Y/N)  ◄── Flag                          │
│  is_online_order       (Y/N)  ◄── Flag                          │
│  payment_type          (3 values)                               │
│  shipping_method       (4 values)                               │
│  order_amount                                                   │
└─────────────────────────────────────────────────────────────────┘
```

#### After Junk Dimension

```
┌───────────────────────────────────┐
│        DIM_ORDER_FLAGS            │
│        (Junk Dimension)           │
├───────────────────────────────────┤
│  order_flags_key (PK)             │
│  is_rush_order                    │
│  is_gift_wrapped                  │
│  is_online_order                  │       ┌─────────────────────────────┐
│  payment_type                     │       │    FACT_ORDERS (After)      │
│  shipping_method                  │       ├─────────────────────────────┤
└──────────────────┬────────────────┘       │  order_key                  │
                   │                        │  date_key (FK)              │
                   │                        │  customer_key (FK)          │
                   └───────────────────────►│  order_flags_key (FK)       │
                                            │  order_amount               │
                                            └─────────────────────────────┘
```

#### Benefits of Junk Dimensions

- **Narrower fact tables** (fewer columns)
- **Fewer foreign keys** to manage
- **Cleaner data model**
- **Pre-computed combinations** for efficient filtering

#### Junk Dimension Size

The maximum number of rows = product of all cardinalities:

```
is_rush_order (2) × is_gift_wrapped (2) × is_online_order (2) × 
payment_type (3) × shipping_method (4) = 96 possible combinations
```

---

## Building a Star Schema with Python

Let's build a complete star schema for a retail sales data warehouse using Python and pandas.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("Building a Star Schema for Retail Sales Data Warehouse")
print("=" * 55)

### 1. Create Date Dimension (Conformed Dimension)

In [ ]:
def create_date_dimension(start_date: str, end_date: str) -> pd.DataFrame:
    """
    Create a comprehensive date dimension table.
    This is a conformed dimension used across multiple fact tables.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    dim_date = pd.DataFrame({
        'date_key': [int(d.strftime('%Y%m%d')) for d in date_range],
        'full_date': date_range,
        'day_of_week': date_range.day_name(),
        'day_of_month': date_range.day,
        'day_of_year': date_range.dayofyear,
        'week_of_year': date_range.isocalendar().week.values,
        'month_number': date_range.month,
        'month_name': date_range.month_name(),
        'quarter': date_range.quarter,
        'year': date_range.year,
        'is_weekend': date_range.dayofweek >= 5,
        'fiscal_year': np.where(date_range.month >= 7, date_range.year + 1, date_range.year),
        'fiscal_quarter': np.where(date_range.month >= 7, 
                                   ((date_range.month - 7) // 3) + 1, 
                                   ((date_range.month + 5) // 3))
    })
    
    return dim_date

# Create date dimension for 2 years
dim_date = create_date_dimension('2024-01-01', '2025-12-31')
print(f"Date Dimension: {len(dim_date)} rows")
dim_date.head(10)

### 2. Create Product Dimension

In [ ]:
def create_product_dimension() -> pd.DataFrame:
    """
    Create product dimension with hierarchy (Category > Subcategory > Product).
    Includes SCD Type 2 columns for tracking changes.
    """
    products = [
        # Electronics > Computers
        {'product_id': 'P001', 'product_name': 'Laptop Pro 15', 'category': 'Electronics', 
         'subcategory': 'Computers', 'brand': 'TechCorp', 'unit_cost': 800, 'unit_price': 1299},
        {'product_id': 'P002', 'product_name': 'Desktop Workstation', 'category': 'Electronics', 
         'subcategory': 'Computers', 'brand': 'TechCorp', 'unit_cost': 1200, 'unit_price': 1899},
        # Electronics > Phones
        {'product_id': 'P003', 'product_name': 'Smartphone X', 'category': 'Electronics', 
         'subcategory': 'Phones', 'brand': 'MobileMax', 'unit_cost': 400, 'unit_price': 799},
        {'product_id': 'P004', 'product_name': 'Tablet Pro', 'category': 'Electronics', 
         'subcategory': 'Phones', 'brand': 'MobileMax', 'unit_cost': 300, 'unit_price': 599},
        # Home & Garden > Furniture
        {'product_id': 'P005', 'product_name': 'Office Chair', 'category': 'Home & Garden', 
         'subcategory': 'Furniture', 'brand': 'ComfortPlus', 'unit_cost': 150, 'unit_price': 349},
        {'product_id': 'P006', 'product_name': 'Standing Desk', 'category': 'Home & Garden', 
         'subcategory': 'Furniture', 'brand': 'ComfortPlus', 'unit_cost': 400, 'unit_price': 699},
        # Home & Garden > Appliances
        {'product_id': 'P007', 'product_name': 'Coffee Maker Pro', 'category': 'Home & Garden', 
         'subcategory': 'Appliances', 'brand': 'BrewMaster', 'unit_cost': 80, 'unit_price': 159},
        {'product_id': 'P008', 'product_name': 'Air Purifier', 'category': 'Home & Garden', 
         'subcategory': 'Appliances', 'brand': 'CleanAir', 'unit_cost': 120, 'unit_price': 249},
        # Sports > Fitness
        {'product_id': 'P009', 'product_name': 'Yoga Mat Premium', 'category': 'Sports', 
         'subcategory': 'Fitness', 'brand': 'FitLife', 'unit_cost': 20, 'unit_price': 49},
        {'product_id': 'P010', 'product_name': 'Dumbbell Set', 'category': 'Sports', 
         'subcategory': 'Fitness', 'brand': 'FitLife', 'unit_cost': 60, 'unit_price': 129},
    ]
    
    dim_product = pd.DataFrame(products)
    dim_product['product_key'] = range(1, len(dim_product) + 1)
    
    # SCD Type 2 columns
    dim_product['effective_date'] = pd.Timestamp('2024-01-01')
    dim_product['expiration_date'] = pd.Timestamp('9999-12-31')
    dim_product['is_current'] = True
    
    # Reorder columns
    cols = ['product_key', 'product_id', 'product_name', 'category', 'subcategory', 
            'brand', 'unit_cost', 'unit_price', 'effective_date', 'expiration_date', 'is_current']
    
    return dim_product[cols]

dim_product = create_product_dimension()
print(f"Product Dimension: {len(dim_product)} rows")
dim_product

### 3. Create Customer Dimension (with SCD Type 2 Example)

In [ ]:
def create_customer_dimension() -> pd.DataFrame:
    """
    Create customer dimension with SCD Type 2 history.
    Shows example of a customer who changed address.
    """
    customers = [
        # Current customers
        {'customer_key': 1, 'customer_id': 'C001', 'customer_name': 'Alice Johnson', 
         'email': 'alice@email.com', 'city': 'New York', 'state': 'NY', 'country': 'USA',
         'segment': 'Premium', 'effective_date': '2024-01-01', 'expiration_date': '9999-12-31', 'is_current': True},
        
        # Customer with address change history (SCD Type 2)
        {'customer_key': 2, 'customer_id': 'C002', 'customer_name': 'Bob Smith', 
         'email': 'bob@email.com', 'city': 'Boston', 'state': 'MA', 'country': 'USA',
         'segment': 'Regular', 'effective_date': '2024-01-01', 'expiration_date': '2024-06-30', 'is_current': False},
        {'customer_key': 3, 'customer_id': 'C002', 'customer_name': 'Bob Smith', 
         'email': 'bob@email.com', 'city': 'Chicago', 'state': 'IL', 'country': 'USA',  # Moved!
         'segment': 'Regular', 'effective_date': '2024-07-01', 'expiration_date': '9999-12-31', 'is_current': True},
        
        {'customer_key': 4, 'customer_id': 'C003', 'customer_name': 'Carol White', 
         'email': 'carol@email.com', 'city': 'Los Angeles', 'state': 'CA', 'country': 'USA',
         'segment': 'Premium', 'effective_date': '2024-01-01', 'expiration_date': '9999-12-31', 'is_current': True},
        
        {'customer_key': 5, 'customer_id': 'C004', 'customer_name': 'David Brown', 
         'email': 'david@email.com', 'city': 'Seattle', 'state': 'WA', 'country': 'USA',
         'segment': 'Regular', 'effective_date': '2024-01-01', 'expiration_date': '9999-12-31', 'is_current': True},
        
        {'customer_key': 6, 'customer_id': 'C005', 'customer_name': 'Eva Martinez', 
         'email': 'eva@email.com', 'city': 'Miami', 'state': 'FL', 'country': 'USA',
         'segment': 'Budget', 'effective_date': '2024-01-01', 'expiration_date': '9999-12-31', 'is_current': True},
    ]
    
    dim_customer = pd.DataFrame(customers)
    dim_customer['effective_date'] = pd.to_datetime(dim_customer['effective_date'])
    dim_customer['expiration_date'] = pd.to_datetime(dim_customer['expiration_date'])
    
    return dim_customer

dim_customer = create_customer_dimension()
print(f"Customer Dimension: {len(dim_customer)} rows (including SCD Type 2 history)")
print("\nNotice customer C002 (Bob Smith) has 2 rows showing address change:")
dim_customer

### 4. Create Store Dimension

In [ ]:
def create_store_dimension() -> pd.DataFrame:
    """
    Create store dimension with location hierarchy.
    """
    stores = [
        {'store_key': 1, 'store_id': 'S001', 'store_name': 'Downtown Flagship', 
         'store_type': 'Flagship', 'city': 'New York', 'state': 'NY', 'region': 'Northeast', 'sq_feet': 50000},
        {'store_key': 2, 'store_id': 'S002', 'store_name': 'Mall Location', 
         'store_type': 'Mall', 'city': 'Los Angeles', 'state': 'CA', 'region': 'West', 'sq_feet': 25000},
        {'store_key': 3, 'store_id': 'S003', 'store_name': 'Suburban Store', 
         'store_type': 'Standard', 'city': 'Chicago', 'state': 'IL', 'region': 'Midwest', 'sq_feet': 30000},
        {'store_key': 4, 'store_id': 'S004', 'store_name': 'Outlet Center', 
         'store_type': 'Outlet', 'city': 'Miami', 'state': 'FL', 'region': 'Southeast', 'sq_feet': 20000},
        {'store_key': 5, 'store_id': 'S005', 'store_name': 'Tech Hub Store', 
         'store_type': 'Flagship', 'city': 'Seattle', 'state': 'WA', 'region': 'West', 'sq_feet': 35000},
        {'store_key': 0, 'store_id': 'ONLINE', 'store_name': 'Online Store', 
         'store_type': 'E-Commerce', 'city': 'N/A', 'state': 'N/A', 'region': 'Online', 'sq_feet': 0},
    ]
    
    return pd.DataFrame(stores)

dim_store = create_store_dimension()
print(f"Store Dimension: {len(dim_store)} rows")
dim_store

### 5. Create Junk Dimension (Order Flags)

In [ ]:
def create_order_flags_junk_dimension() -> pd.DataFrame:
    """
    Create junk dimension combining low-cardinality order flags.
    Generates all possible combinations.
    """
    from itertools import product
    
    # Define all possible values for each flag
    is_rush_order = ['Y', 'N']
    is_gift_wrapped = ['Y', 'N']
    payment_type = ['Credit Card', 'Debit Card', 'PayPal']
    shipping_method = ['Standard', 'Express', 'Overnight', 'Store Pickup']
    
    # Generate all combinations
    combinations = list(product(is_rush_order, is_gift_wrapped, payment_type, shipping_method))
    
    dim_order_flags = pd.DataFrame(combinations, 
                                   columns=['is_rush_order', 'is_gift_wrapped', 
                                           'payment_type', 'shipping_method'])
    
    dim_order_flags['order_flags_key'] = range(1, len(dim_order_flags) + 1)
    
    # Reorder columns
    cols = ['order_flags_key', 'is_rush_order', 'is_gift_wrapped', 'payment_type', 'shipping_method']
    return dim_order_flags[cols]

dim_order_flags = create_order_flags_junk_dimension()
print(f"Order Flags Junk Dimension: {len(dim_order_flags)} rows")
print(f"  (2 × 2 × 3 × 4 = {2*2*3*4} combinations)")
dim_order_flags.head(12)

### 6. Create Transactional Fact Table (Sales)

In [ ]:
def create_fact_sales(dim_date: pd.DataFrame, 
                      dim_product: pd.DataFrame,
                      dim_customer: pd.DataFrame,
                      dim_store: pd.DataFrame,
                      dim_order_flags: pd.DataFrame,
                      num_transactions: int = 1000) -> pd.DataFrame:
    """
    Create transactional fact table for sales.
    Each row represents one line item in a sales transaction.
    """
    
    # Get valid keys from dimensions
    date_keys = dim_date['date_key'].tolist()
    product_keys = dim_product[dim_product['is_current']]['product_key'].tolist()
    customer_keys = dim_customer[dim_customer['is_current']]['customer_key'].tolist()
    store_keys = dim_store['store_key'].tolist()
    order_flags_keys = dim_order_flags['order_flags_key'].tolist()
    
    # Generate random transactions
    transactions = []
    
    for i in range(num_transactions):
        date_key = random.choice(date_keys)
        product_key = random.choice(product_keys)
        customer_key = random.choice(customer_keys)
        store_key = random.choice(store_keys)
        order_flags_key = random.choice(order_flags_keys)
        
        # Get product price
        product = dim_product[dim_product['product_key'] == product_key].iloc[0]
        unit_price = product['unit_price']
        unit_cost = product['unit_cost']
        
        quantity = random.randint(1, 5)
        discount_pct = random.choice([0, 0, 0, 0.05, 0.10, 0.15, 0.20])
        
        gross_amount = quantity * unit_price
        discount_amount = round(gross_amount * discount_pct, 2)
        net_amount = round(gross_amount - discount_amount, 2)
        cost_amount = quantity * unit_cost
        profit_amount = round(net_amount - cost_amount, 2)
        
        transactions.append({
            'sale_key': i + 1,
            'date_key': date_key,
            'product_key': product_key,
            'customer_key': customer_key,
            'store_key': store_key,
            'order_flags_key': order_flags_key,
            'quantity_sold': quantity,
            'unit_price': unit_price,
            'discount_amount': discount_amount,
            'net_sales_amount': net_amount,
            'cost_amount': cost_amount,
            'profit_amount': profit_amount
        })
    
    return pd.DataFrame(transactions)

# Generate 1000 sales transactions
fact_sales = create_fact_sales(dim_date, dim_product, dim_customer, 
                                dim_store, dim_order_flags, num_transactions=1000)

print(f"Fact Sales (Transactional): {len(fact_sales)} rows")
fact_sales.head(10)

### 7. Visualize the Star Schema

In [ ]:
def print_star_schema_diagram():
    """
    Print ASCII diagram of our star schema.
    """
    diagram = """
    ╔═══════════════════════════════════════════════════════════════════════════════════════╗
    ║                           RETAIL SALES STAR SCHEMA                                    ║
    ╠═══════════════════════════════════════════════════════════════════════════════════════╣
    ║                                                                                       ║
    ║    ┌───────────────────┐                              ┌───────────────────┐           ║
    ║    │    DIM_DATE       │                              │   DIM_PRODUCT     │           ║
    ║    │   (Conformed)     │                              │                   │           ║
    ║    ├───────────────────┤                              ├───────────────────┤           ║
    ║    │ date_key (PK)     │                              │ product_key (PK)  │           ║
    ║    │ full_date         │                              │ product_id        │           ║
    ║    │ day_of_week       │                              │ product_name      │           ║
    ║    │ month_name        │                              │ category          │           ║
    ║    │ quarter           │                              │ subcategory       │           ║
    ║    │ fiscal_year       │                              │ brand             │           ║
    ║    │ is_weekend        │                              │ unit_price        │           ║
    ║    └─────────┬─────────┘                              └─────────┬─────────┘           ║
    ║              │                                                  │                     ║
    ║              │         ┌────────────────────────────────┐       │                     ║
    ║              │         │        FACT_SALES              │       │                     ║
    ║              │         │     (Transactional)            │       │                     ║
    ║              │         ├────────────────────────────────┤       │                     ║
    ║              └────────►│ sale_key (PK)                  │◄──────┘                     ║
    ║                        │ date_key (FK)                  │                             ║
    ║              ┌────────►│ product_key (FK)               │◄──────┐                     ║
    ║              │         │ customer_key (FK)              │       │                     ║
    ║              │         │ store_key (FK)                 │       │                     ║
    ║              │         │ order_flags_key (FK)           │       │                     ║
    ║              │         │ ────────────────────────────── │       │                     ║
    ║              │         │ quantity_sold                  │       │                     ║
    ║              │         │ unit_price                     │       │                     ║
    ║              │         │ discount_amount                │       │                     ║
    ║              │         │ net_sales_amount               │       │                     ║
    ║              │         │ cost_amount                    │       │                     ║
    ║              │         │ profit_amount                  │       │                     ║
    ║              │         └────────────────────────────────┘       │                     ║
    ║              │                     ▲           ▲                │                     ║
    ║              │                     │           │                │                     ║
    ║    ┌─────────┴─────────┐ ┌─────────┴─────────┐ │   ┌────────────┴────────┐            ║
    ║    │   DIM_CUSTOMER    │ │    DIM_STORE      │ │   │  DIM_ORDER_FLAGS    │            ║
    ║    │    (SCD Type 2)   │ │                   │ │   │  (Junk Dimension)   │            ║
    ║    ├───────────────────┤ ├───────────────────┤ │   ├─────────────────────┤            ║
    ║    │ customer_key (PK) │ │ store_key (PK)    │ │   │ order_flags_key (PK)│            ║
    ║    │ customer_id       │ │ store_id          │ │   │ is_rush_order       │            ║
    ║    │ customer_name     │ │ store_name        │ │   │ is_gift_wrapped     │            ║
    ║    │ city, state       │ │ store_type        │ │   │ payment_type        │            ║
    ║    │ segment           │ │ region            │ │   │ shipping_method     │            ║
    ║    │ effective_date    │ │ sq_feet           │ │   └─────────────────────┘            ║
    ║    │ expiration_date   │ └───────────────────┘ │                                      ║
    ║    │ is_current        │                       │                                      ║
    ║    └───────────────────┘                       │                                      ║
    ║                                                │                                      ║
    ╚════════════════════════════════════════════════╪══════════════════════════════════════╝
    """
    print(diagram)

print_star_schema_diagram()

### 8. Sample Analytical Queries

In [ ]:
# Merge fact and dimension tables for analysis
def build_denormalized_view():
    """
    Create a denormalized view by joining fact table with all dimensions.
    """
    # Start with fact table
    sales_analysis = fact_sales.copy()
    
    # Join with date dimension
    sales_analysis = sales_analysis.merge(
        dim_date[['date_key', 'full_date', 'month_name', 'quarter', 'year', 'is_weekend']],
        on='date_key'
    )
    
    # Join with product dimension (current rows only)
    sales_analysis = sales_analysis.merge(
        dim_product[dim_product['is_current']][['product_key', 'product_name', 'category', 'subcategory', 'brand']],
        on='product_key'
    )
    
    # Join with customer dimension (current rows only)
    sales_analysis = sales_analysis.merge(
        dim_customer[dim_customer['is_current']][['customer_key', 'customer_name', 'city', 'segment']],
        on='customer_key'
    )
    
    # Join with store dimension
    sales_analysis = sales_analysis.merge(
        dim_store[['store_key', 'store_name', 'store_type', 'region']],
        on='store_key'
    )
    
    # Join with junk dimension
    sales_analysis = sales_analysis.merge(
        dim_order_flags,
        on='order_flags_key'
    )
    
    return sales_analysis

sales_analysis = build_denormalized_view()
print(f"Denormalized Sales Analysis View: {len(sales_analysis)} rows, {len(sales_analysis.columns)} columns")
sales_analysis.head()

In [ ]:
# Query 1: Sales by Category and Quarter
print("\n📊 QUERY 1: Sales by Product Category and Quarter")
print("=" * 50)

category_quarter_sales = sales_analysis.groupby(['category', 'quarter']).agg({
    'net_sales_amount': 'sum',
    'profit_amount': 'sum',
    'quantity_sold': 'sum'
}).round(2)

category_quarter_sales

In [ ]:
# Query 2: Top Customers by Segment
print("\n📊 QUERY 2: Sales by Customer Segment")
print("=" * 50)

segment_analysis = sales_analysis.groupby('segment').agg({
    'net_sales_amount': ['sum', 'mean'],
    'sale_key': 'count',
    'profit_amount': 'sum'
}).round(2)

segment_analysis.columns = ['Total Sales', 'Avg Transaction', 'Num Transactions', 'Total Profit']
segment_analysis.sort_values('Total Sales', ascending=False)

In [ ]:
# Query 3: Junk Dimension Analysis - Payment & Shipping Impact
print("\n📊 QUERY 3: Sales by Payment Type and Shipping Method (Junk Dimension)")
print("=" * 60)

junk_analysis = sales_analysis.groupby(['payment_type', 'shipping_method']).agg({
    'net_sales_amount': 'sum',
    'sale_key': 'count'
}).round(2)

junk_analysis.columns = ['Total Sales', 'Num Transactions']
junk_analysis.sort_values('Total Sales', ascending=False)

In [ ]:
# Query 4: Weekend vs Weekday Sales
print("\n📊 QUERY 4: Weekend vs Weekday Sales Performance")
print("=" * 50)

weekend_analysis = sales_analysis.groupby('is_weekend').agg({
    'net_sales_amount': ['sum', 'mean'],
    'quantity_sold': 'sum',
    'profit_amount': 'sum'
}).round(2)

weekend_analysis.columns = ['Total Sales', 'Avg Sale', 'Units Sold', 'Total Profit']
weekend_analysis.index = ['Weekday', 'Weekend']
weekend_analysis

In [ ]:
# Summary Statistics
print("\n📈 STAR SCHEMA SUMMARY STATISTICS")
print("=" * 50)
print(f"\nDimension Tables:")
print(f"  • DIM_DATE:         {len(dim_date):,} rows")
print(f"  • DIM_PRODUCT:      {len(dim_product):,} rows")
print(f"  • DIM_CUSTOMER:     {len(dim_customer):,} rows (with SCD Type 2)")
print(f"  • DIM_STORE:        {len(dim_store):,} rows")
print(f"  • DIM_ORDER_FLAGS:  {len(dim_order_flags):,} rows (junk dimension)")

print(f"\nFact Table:")
print(f"  • FACT_SALES:       {len(fact_sales):,} rows")

print(f"\nKey Metrics:")
print(f"  • Total Revenue:     ${fact_sales['net_sales_amount'].sum():,.2f}")
print(f"  • Total Profit:      ${fact_sales['profit_amount'].sum():,.2f}")
print(f"  • Total Units Sold:  {fact_sales['quantity_sold'].sum():,}")
print(f"  • Avg Transaction:   ${fact_sales['net_sales_amount'].mean():,.2f}")

---

## Key Takeaways

### 🎯 Core Concepts

| Concept | Key Point |
|---------|----------|
| **Kimball Methodology** | Bottom-up approach building data marts around business processes |
| **Grain** | The most critical decision - defines what one fact row represents |
| **Star Schema** | Denormalized design optimized for query performance |
| **Facts** | Numeric measurements that can be aggregated |
| **Dimensions** | Descriptive context (who, what, where, when, why, how) |

### 📊 Fact Table Types

| Type | Grain | Updates | Best For |
|------|-------|---------|----------|
| **Transactional** | One event | Insert only | Event analysis |
| **Periodic Snapshot** | One entity/period | Insert only | State trending |
| **Accumulating Snapshot** | One process | Updates | Process tracking |

### 🔄 SCD Types Summary

| Type | Strategy | When to Use |
|------|----------|-------------|
| **0** | Never change | Fixed attributes (SSN, birth date) |
| **1** | Overwrite | Data corrections, non-critical changes |
| **2** | New row | Important business attributes needing history |
| **3** | New column | Before/after comparisons |
| **4** | Mini-dimension | Rapidly changing demographics |
| **6** | Hybrid | Need both history and current value |

### 🏗️ Design Best Practices

1. **Always use surrogate keys** - Isolate data warehouse from source system changes
2. **Declare grain first** - Everything else flows from grain
3. **Denormalize dimensions** - Optimize for query performance
4. **Use conformed dimensions** - Enable drill-across queries
5. **Create junk dimensions** - Reduce fact table width for low-cardinality flags
6. **Choose appropriate SCD types** - Match business requirements for each attribute

### 📚 Further Reading

- *The Data Warehouse Toolkit* by Ralph Kimball (the definitive guide)
- *Star Schema: The Complete Reference* by Christopher Adamson
- Kimball Group website: [kimballgroup.com](https://www.kimballgroup.com/)

---

**Remember:** The star schema is not just a modeling technique—it's a **business-oriented approach** to organizing data that makes analytics intuitive and fast. The dimension tables provide the natural language of the business, while fact tables capture the measurements that drive decision-making.